## Deliverable 2. Create a Customer Travel Destinations Map.

In [54]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [61]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("D:\\UofT\\Week6\\World_Weather_Analysis\\Weather_Database\\WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Majene,-3.5403,118.9707,26.03,85,91,0.40,overcast clouds,ID
1,1,Souillac,-20.5167,57.5167,26.21,78,40,3.60,scattered clouds,MU
2,2,Jamestown,42.0970,-79.2353,6.12,78,100,4.63,overcast clouds,US
3,3,Saskylakh,71.9167,114.0833,-24.06,100,45,1.86,scattered clouds,RU
4,4,Oppdal,62.5943,9.6912,-12.94,99,77,1.94,broken clouds,NO


In [66]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
mintemp = float(input("Inform the minimum Temp preferable:"))
maxtemp = float(input("Inform the maximum Temp preferable:"))

Inform the minimum Temp preferable:28
Inform the maximum Temp preferable:35


In [67]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_temp_df = city_data_df.loc[(city_data_df["Max Temp"] >= mintemp) & (city_data_df["Max Temp"] <= maxtemp)]
city_data_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
16,16,Avarua,-21.2078,-159.7750,30.03,70,20,5.14,few clouds,CK
35,35,Ugoofaaru,5.6667,73.0000,28.82,65,67,1.09,broken clouds,MV
59,59,Noumea,-22.2763,166.4572,29.99,99,97,9.09,light rain,NC
74,74,Vanimo,-2.6741,141.3028,28.63,71,100,0.92,overcast clouds,PG
96,96,Alofi,-19.0595,-169.9187,28.94,65,75,7.20,broken clouds,NU
...,...,...,...,...,...,...,...,...,...,...
671,671,Maumere,-8.6199,122.2111,28.04,76,86,1.66,light rain,ID
672,672,Sarangani,5.4033,125.4636,28.07,79,100,9.85,overcast clouds,PH
677,677,Banamba,13.5500,-7.4500,30.64,13,23,4.32,few clouds,ML
689,689,Voh,-20.9667,164.7000,28.84,69,100,6.10,overcast clouds,NC


In [68]:
# 4a. Determine if there are any empty rows.
city_data_temp_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                0
dtype: int64

In [69]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=city_data_temp_df.dropna()


In [70]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Avarua,CK,30.03,few clouds,-21.2078,-159.7750,
35,Ugoofaaru,MV,28.82,broken clouds,5.6667,73.0000,
59,Noumea,NC,29.99,light rain,-22.2763,166.4572,
74,Vanimo,PG,28.63,overcast clouds,-2.6741,141.3028,
96,Alofi,NU,28.94,broken clouds,-19.0595,-169.9187,
103,Vaini,TO,28.09,broken clouds,-21.2000,-175.2000,
113,Mandiana,GN,30.01,broken clouds,10.6333,-8.6833,
116,Neiafu,TO,30.38,scattered clouds,-18.6500,-173.9833,
121,Kahului,US,29.01,few clouds,20.8947,-156.4700,
128,Butte,US,31.12,broken clouds,39.6499,-121.5677,


In [71]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found - Skip")
        

Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip
Hotel not found - Skip


In [79]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"].astype(bool)]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Avarua,CK,30.03,few clouds,-21.2078,-159.7750,Paradise Inn
35,Ugoofaaru,MV,28.82,broken clouds,5.6667,73.0000,Ungoofaaru Inn
59,Noumea,NC,29.99,light rain,-22.2763,166.4572,Hôtel Le Lagon
74,Vanimo,PG,28.63,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
96,Alofi,NU,28.94,broken clouds,-19.0595,-169.9187,Taloa Heights
103,Vaini,TO,28.09,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
113,Mandiana,GN,30.01,broken clouds,10.6333,-8.6833,Motels rouzo
116,Neiafu,TO,30.38,scattered clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
121,Kahului,US,29.01,few clouds,20.8947,-156.4700,Maui Seaside Hotel
128,Butte,US,31.12,broken clouds,39.6499,-121.5677,Meadowbrook Oaks Mobile Home Park


In [73]:
# 8a. Create the output File (CSV)
output_data_file="D:\\UofT\\Week6\\World_Weather_Analysis\\Vacation_Search\\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [77]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and Temp Max {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [78]:
# 11a. Add a marker layer for each city to the map. 
locations =clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))